<a href="https://colab.research.google.com/github/knoel99/learn_cuda/blob/master/01_easier_intro_to_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An even easier introduction to CUDA

Source: https://developer.nvidia.com/blog/even-easier-introduction-cuda/

# Addition of two arrays